## Import Libraries

In [2]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pyperclip

import pandas as pd
import time

### Clipboard copy

In [3]:
def clipboard_input(driver, user_input):
    # save existing clipboard data
    existing_clipboard = pyperclip.paste()

    # copy user_input to clipboard. then paste it
    pyperclip.copy(user_input)
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

    # recover the original clipboard data
    pyperclip.copy(existing_clipboard)  
    time.sleep(1)

### Load user info

In [4]:
def load_user_info():
    f = open('user_info.txt', 'r')
    user_info = f.readlines()
    f.close()
    
    return user_info[0][:-1], user_info[1][:-1]

### Naver Login

In [5]:
def login_naver(driver, user_id, user_pw):
    # naver login page
    driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/aside/')

    # input id 
    driver.find_element_by_xpath('//*[@id="id"]').click()
    clipboard_input(driver, user_id)

    # input pw
    driver.find_element_by_xpath('//*[@id="pw"]').click()
    clipboard_input(driver, user_pw)

    # click login btn
    driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
    time.sleep(1)

    # click 
    driver.find_element_by_xpath('//span[@class="btn_cancel"]').click()

### Enter Details to search in Joonggonara

In [6]:
def get_idx_of_selected_option(option_list, input_text):
    for idx, option in enumerate(option_list):
        if option.text == input_text:
            return idx
    return -1

In [7]:
def enter_details(driver, search_option, item, detail_option):    
    # show 50 items
    driver.find_element_by_xpath('//div[@id="listSizeSelectDiv"]').click()
    time.sleep(1)
    listSize_list = driver.find_elements_by_xpath('//div[@id="listSizeSelectDiv"]/ul/li')
    listSize_list[-1].click()
    time.sleep(1)
    
    # set search options
    driver.find_element_by_xpath('//div[@id="searchOptionSelectDiv"]').click()
    searchBoard_list = driver.find_elements_by_xpath('//div[@id="searchOptionSelectDiv"]/ul/li')
    searchBoard_list[get_idx_of_selected_option(searchBoard_list, search_option['board'])].click()
    time.sleep(1)
    
    driver.find_element_by_xpath('//div[@id="divSearchDateTop"]').click()
    searchDate_list = driver.find_elements_by_xpath('//div[@id="divSearchDateTop"]/ul/li')
    searchDate_list[get_idx_of_selected_option(searchDate_list, search_option['period'])].click()
    
    driver.find_element_by_xpath('//div[@id="divSearchMenuTop"]').click()
    searchMenu_list = driver.find_elements_by_xpath('//div[@id="divSearchMenuTop"]/ul/li')
    searchMenu_list[get_idx_of_selected_option(searchMenu_list, search_option['menu'])].click()
    
    driver.find_element_by_xpath('//div[@id="divSearchByTop"]').click()
    searchBy_list = driver.find_elements_by_xpath('//div[@id="divSearchByTop"]/ul/li')
    searchBy_list[get_idx_of_selected_option(searchBy_list, search_option['by'])].click()
    
    # enter item
    driver.find_element_by_xpath('//input[@placeholder="검색어를 입력해주세요"]').send_keys(item)
    
    # click detail search btn
    driver.find_element_by_xpath('//*[@id="detailSearchBtn"]').click()

    # enter detail options
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 모두 포함"]').send_keys(detail_option['keywords_and'])
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 제외"]').send_keys(detail_option['keywords_not'])
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 중 1개 이상 포함"]').send_keys(detail_option['keywords_or'])
    driver.find_element_by_xpath('//input[@placeholder="다음 어절, 어구 정확히 일치"]').send_keys(detail_option['sentence'])
    
    
    # search
    driver.find_element_by_xpath('//form[@name="frmSearchTop"]/div[@class="input_search_area"]/button[@class="btn-search-green"]').click()
    
    time.sleep(1)

### Get Product Info

In [8]:
def get_post_info(driver, href):
    driver.get(href)
    time.sleep(1)
    driver.switch_to.frame('cafe_main')
    soup = bs(driver.page_source, 'html.parser')

    # post_title
    title = soup.select('div.tit-box span.b')[0].get_text()
    
    # posted_cost
    try:
        cost = soup.select('span.cost')[0].get_text()
    except:
        cost = 0

    # merge contents to single text
    content_tags = soup.select('#tbody')[0].select('p')
    content = ' '.join([ tags.get_text() for tags in content_tags ])
    
    time.sleep(1)

    return {'cost':cost, 'title' : title, 'content' : content}

#### Run ChromeDriver

In [9]:
driver = webdriver.Chrome()
driver.implicitly_wait(2)

#### Login Naver

In [10]:
user_id, user_pw = load_user_info()
login_naver(driver, user_id, user_pw)

#### Search items from Joonggonara and get boards

In [11]:
# log onto Joonggonara and enter details
driver.get('https://cafe.naver.com/joonggonara?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10050146%26search.searchBy=0')
time.sleep(1)
driver.switch_to.frame(driver.find_element_by_name("cafe_main"))

search_option={
    'board' : "게시글 전체",
    'period' : "전체기간",
    'menu' : "주변기기/악세사리",
    'by' : "제목만"
}

item = "에어팟"

detail_option = {
    'keywords_and' : "미개봉", 
    'keywords_not' : "프로 중고폰", 
    'keywords_or' : "유선", 
    'sentence' : ""
}

enter_details(driver, search_option, item, detail_option)

In [12]:
board_navigator = driver.find_element_by_xpath('//div[@class="prev-next"]')
board_list = board_navigator.find_elements_by_tag_name('a')
board_href = board_list[1].get_attribute('href')[:-1]
board_idx = 0

#### Get Posts Links

In [13]:
post_link_list=[]
while True:
    # change board
    board_idx+=1
    driver.get(board_href+str(board_idx))
    time.sleep(1)
    driver.switch_to.frame(driver.find_element_by_name("cafe_main"))
    
    # get posts
    post_list = driver.find_elements_by_class_name('article')
    for post in post_list:
        post_link_list.append(post.get_attribute('href'))
    
    # check
    print("read {} pages {} posts".format(board_idx, len(post_link_list)))
    
    # terminate condition
    if len(post_list) == 0 or board_idx>1:
        break

read 1 pages 50 posts
read 2 pages 100 posts


#### Get Price Info

In [14]:
price_info=[]
for post_link in post_link_list:
    try:
        post_info = get_post_info(driver, post_link)
        print(post_info['cost'], post_info['title'])
        price_info.append(post_info)
    except:
        print("cannot attach post")

price_info_pd=pd.DataFrame(price_info)
price_info_pd.to_csv("price_info.csv")

140,000원 (미개봉)에어팟2유선 14만팝니다 서울!
130,000원 에어팟 2(유선) 미개봉 삽니다
152,000원 에어팟2세대 유선충전 미개봉 새제품 MV7N2KH/A
150,000원 에어팟 2세대 유선충전(미개봉) 15만 팝니다
150,000원 에어팟/유선/미개봉 새제품 팝니다:) 
149,000원 애플 에어팟 2세대 유선  - 미개봉 새제품 ( MV7N2KH/A )
140,000원 에어팟2(유선) 미개봉 제품 판매합니다.
150,000원 ▶▷애플 에어팟 2세대 유선 충전 모델, MV7N2KH/A 미개봉 새상품 판매합니다.
170,000원 에어팟2 유선충전 미개봉 
155,000원 에어팟 2세대 유선충전 미개봉 팝니다 (택포)
130,000원 [직거래] 에어팟2 유선 충전 미개봉 정품 1대 삽니다 
2,000원 전주) 미개봉 에어팟2 유선모델 팝니다 싸게
150,000원 울산 에어팟2세대 유선 미개봉 울산직거래가능
140,000원 (급) 에어팟2 (유선) 미개봉 팜
150,000원 에어팟 2세대 유선충전 미개봉 판매합니다.
150,000원 에어팟2세대 유선 정품 미개봉 제품팝니다. 광주
150,000원 미개봉 유선 에어팟2 + 새 케이스
150,000원 에어팟 2세대 유선충전 애플코리아 정품 미개봉 팝니다.
150,000원 에어팟2 유선/무선 미개봉 새상품 판매
140,000원 에어팟2 유선 (미개봉)판매합니다.
150,000원 에어팟2세대 유선 국내정식 미개봉  15만
145,000원 애플코리아 정품)애플 에어팟2세대 유선 미개봉 새상품 팝니다 언주역 직거래 
145,000원 애플코리아 정품)애플 에어팟2세대 유선 미개봉 새상품 팝니다 언주역 직거래 
145,000원 애플코리아 정품)애플 에어팟2세대 유선 미개봉 새상품 팝니다 언주역 직거래 
2,222원 에어팟 2세대 미개봉 유선  국내정품 팝니다
140,000원 에어팟2세대 (유선) 미개봉 새상품 팝니다.
152,000원 에어팟2세대 유선충전 미개봉 새제품 MV7N2KH/A
160,000원 에어팟